# 物体检测评价指标--mAP(mean Average Precision) 
译自：https://medium.com/@jonathan_hui/map-mean-average-precision-for-object-detection-45c121a31173

mAP是用于评价诸如Faster R-CNN,SSD等物体检测器准确率的指标。它指不同召回率下最大精度的平均值。这听起来复杂但实际上结合例子来看的话非常简单。在此之前，我们先回顾一下什么叫精准率，召回率以及IoU。

### 精准率和召回率

**精准率**用于衡量预测结果的准确性，即正确的预测结果占所有预测结果的比例。<br>
**召回率**用于衡量正确预测结果的好坏，即正确的预测结果占所有正确结果的比例。例如，我们的K个预测结果只找出了80%的正确结果。
下面是数学定义：<br>
$$TP=True positive$$
$$TN=True negative$$
$$FP=False positive$$
$$FN=False negative$$
$$Precision=\frac{TP}{(TP+FP)}$$
$$Recall=\frac{TP}{TP+FN}$$

对于癌症测试来说：<br>
$$Precision=\frac{TP}{total positive results}$$
$$Recall=\frac{TP}{total cancer cases}$$

### IoU(交并比)
交并比衡量两个区域重叠的多少，可以用于衡量物体检测结果与真实值（真实的物体边界）匹配的好坏。
![avatar](https://cdn-images-1.medium.com/max/1600/1*FrmKLxCtkokDC3Yr1wc70w.png)

### AP
我们可以通过一个简单的例子来说明平均准确率（AP）的计算。我们的数据集包含5个苹果。我们收集了模型关于苹果的所有预测结果并且根据置信水平对这些结果排序（从最高置信度到最低置信度）。第二列表明预测结果是否正确。如果跟真实值匹配并且IoU>=0.5表示正确。<br>

|Rank|Correct?|Precision|Recall|
|:-----|:-----|:-----|:-----|
|1|True|1.0|0.2|
|2|True|1.0|0.4|
|3|False|0.67|0.4|
|4|False|0.5|0.4|
|5|False|0.4|0.4|
|6|True|0.5|0.6|
|7|True|0.57|0.8|
|8|False|0.5|0.8|
|9|False|0.44|0.8|
|10|True|0.5|1.0|

让我们计算Rank等于3这一行的精确率和召回率。<br>
精确率是TP占预测结果的比例=2/3=0.67.<br>
召回率是TP占所有正例的比例=2/5=0.4.<br>
可以发现随着测试次数的增加，召回率在上升而精确率上下波动，精确率和召回率构成的曲线如下：<br>
![avater](https://cdn-images-1.medium.com/max/1600/1*VenTq4IgxjmIpOXWdFb-jg.png)

AP可以看做是寻找精确率-召回率曲线（橙色曲线）下方的区域.我们来用一种不那么直接的方式解释这个计算过程。<br>
我们重新绘制了召回率从0到1.0下的曲线，并且将精确率替换为大于等于当前召回率时的最大精确率。公式为：<br>
$$p_{interp}{(r)}=\max_{\widetilde{r} >= r}p(\widetilde{r})$$

事实上，这个公式可视化起来非常简单。我们可以轻松的找到右侧的最高值。<br>
![avater](https://cdn-images-1.medium.com/max/1600/1*zqTL1KW1gwzion9jY8SjHA.png)

AP(平均精确率)可以计算为这11个召回率对应的最大精确率的平均：<br>
$$AP=\frac{1}{11}\times(AP_r{(0)}+AP_r{(0.1)}+...+AP_r{(1.0)})$$
对于这个例子来说，可以轻易找到绿色曲线下面的区域并且除以11得到AP。下面是更加精确的定义。<br>
$$AP=\frac{1}{11}\sum_{r\in\{0.0,...,1.0\}}{AP_r} \\where \ p_{interp}{(r)}=\max_{\widetilde{r} >= r}p(\widetilde{r})$$

pinterp(0.7)的计算可以描述如下：<br>
![avater](https://cdn-images-1.medium.com/max/1600/1*Va8bCUvxq9LQSbolwNMdcQ.jpeg)

在本例中，AP=(5x1.0+4x0.57+2x0.5)/11
![avater](https://cdn-images-1.medium.com/max/1600/1*fnUHMoewu2ivpB6FQUXPxQ.jpeg)

mAP指的就是所有类别上的平均。在很多数据集中用AP代指mAP。

### 在PASCAL VOC challenge中的AP
PASCAL VOC是是一个非常有名的物体检测数据集。在PASCAL VOC challenge中，如果IoU>0.5表明预测是正确的。然而，如果同一个物体被多次检测出，则令第一个预测为正，其他结果为负。PASCAL VOC的mAP跟上面讲到的AP计算一样。

### COCO mAP
最新的研究论文倾向于给出在COCO数据集上的结果。对于COCO，AP是在多个IoU上的平均。AP@[.5：.95]表示IoU在0.5到0.95之间所有结果取平均，步长是0.05。对于COCO比赛，AP是在80个种类中的10个不同级别的IoU下取平均（AP@[.50:.05:.95]）。<br>
下面是YOLOv3模型的AP结果。<br>
![avater](https://cdn-images-1.medium.com/max/1600/1*09w7--mzDQ3Nqd81t-XOjw.png)
其中mAP@.75表示在IoU=0.75时的mAP。